In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import zarr
import gcsfs

xr.set_options(display_style='html')
%matplotlib inline
%config InlineBackend.figure_format = 'retina' 

In [6]:
df = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')
df_ta = df.query("activity_id=='CMIP' & table_id == 'Amon' & variable_id == 'psl' & experiment_id == 'historical'")


       activity_id       institution_id      source_id experiment_id  \
983           CMIP            NOAA-GFDL      GFDL-ESM4    historical   
1769          CMIP            NOAA-GFDL      GFDL-ESM4    historical   
6238          CMIP            NOAA-GFDL       GFDL-CM4    historical   
22134         CMIP                 IPSL   IPSL-CM6A-LR    historical   
22281         CMIP                 IPSL   IPSL-CM6A-LR    historical   
...            ...                  ...            ...           ...   
522933        CMIP                  MRI     MRI-ESM2-0    historical   
523409        CMIP                  MRI     MRI-ESM2-0    historical   
523707        CMIP                 CMCC   CMCC-CM2-SR5    historical   
523724        CMIP                 CMCC   CMCC-CM2-SR5    historical   
523767        CMIP  EC-Earth-Consortium  EC-Earth3-Veg    historical   

       member_id table_id variable_id grid_label  \
983     r3i1p1f1     Amon         psl        gr1   
1769    r2i1p1f1     Amon      

In [7]:
# this only needs to be created once
gcs = gcsfs.GCSFileSystem(token='anon')


In [8]:
def ensemble_splitter(row):
  member_id = row.member_id
  member_id_split = member_id.split('i')
  return(member_id_split[0], member_id_split[1])

df_ta[['realization', 'other']] = df_ta.apply(ensemble_splitter, axis=1, result_type='expand')
df_ta_new_justrealizations = df_ta[df_ta['other'] == '1p1f1']
print(len(df_ta_new_justrealizations))

434


/home/disk/p/aodhan/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [34]:
import os

path_to_store_data = '/home/disk/pna2/aodhan/CMIP6/historical_monthly_tas_google'
for source_id in df_ta_new_justrealizations.source_id.unique():
  one_model_ds = df_ta_new_justrealizations[df_ta_new_justrealizations['source_id'] == source_id]
  if len(one_model_ds) >= 10:    
    for i in range(0, len(one_model_ds)):
      zstore = one_model_ds.zstore.values[i]

      # create a mutable-mapping-style interface to the store
      mapper = gcs.get_mapper(zstore)

      # open it using xarray and zarr
      ds = xr.open_zarr(mapper, consolidated=True)
      model_name = one_model_ds.iloc[i].source_id
      model_directory = os.path.join(path_to_store_data, model_name)
      try:
        os.mkdir(model_directory)
        variant_id = one_model_ds.iloc[i].member_id
        ds.to_netcdf(model_directory + '/' + variant_id)
        print(model_name + ' ' + variant_id)
      except:
        variant_id = one_model_ds.iloc[i].member_id
        ds.to_netcdf(model_directory + '/' + variant_id)
        print(model_name + ' ' + variant_id)
        
  
      

IPSL-CM6A-LR r8i1p1f1
IPSL-CM6A-LR r2i1p1f1
IPSL-CM6A-LR r30i1p1f1
IPSL-CM6A-LR r29i1p1f1
IPSL-CM6A-LR r3i1p1f1
IPSL-CM6A-LR r6i1p1f1
IPSL-CM6A-LR r27i1p1f1
IPSL-CM6A-LR r7i1p1f1
IPSL-CM6A-LR r9i1p1f1
IPSL-CM6A-LR r26i1p1f1
IPSL-CM6A-LR r20i1p1f1
IPSL-CM6A-LR r25i1p1f1
IPSL-CM6A-LR r24i1p1f1
IPSL-CM6A-LR r31i1p1f1
IPSL-CM6A-LR r22i1p1f1
IPSL-CM6A-LR r21i1p1f1
IPSL-CM6A-LR r23i1p1f1
IPSL-CM6A-LR r19i1p1f1
IPSL-CM6A-LR r18i1p1f1
IPSL-CM6A-LR r11i1p1f1
IPSL-CM6A-LR r12i1p1f1
IPSL-CM6A-LR r17i1p1f1
IPSL-CM6A-LR r16i1p1f1
IPSL-CM6A-LR r1i1p1f1
IPSL-CM6A-LR r5i1p1f1
IPSL-CM6A-LR r4i1p1f1
IPSL-CM6A-LR r28i1p1f1
IPSL-CM6A-LR r14i1p1f1
IPSL-CM6A-LR r10i1p1f1
IPSL-CM6A-LR r15i1p1f1
IPSL-CM6A-LR r13i1p1f1
IPSL-CM6A-LR r32i1p1f1
GISS-E2-1-G r2i1p1f1
GISS-E2-1-G r1i1p1f1
GISS-E2-1-G r3i1p1f1
GISS-E2-1-G r5i1p1f1
GISS-E2-1-G r4i1p1f1
GISS-E2-1-G r7i1p1f1
GISS-E2-1-G r6i1p1f1
GISS-E2-1-G r9i1p1f1
GISS-E2-1-G r10i1p1f1
GISS-E2-1-G r8i1p1f1
GISS-E2-1-G r102i1p1f1
GISS-E2-1-G r101i1p1f1
MIROC6 r8i1p1f1
